# Pips and Connection 

In [1]:
%pip install mongoengine
%pip install pycoshark

'%pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'%pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [2]:
import re
from mongoengine import connect
from pycoshark.mongomodels import Project, VCSSystem, Commit, FileAction, Hunk, Refactoring, IssueSystem, Issue, IssueComment, MailingList, Message
from pycoshark.utils import create_mongodb_uri_string
from mongoengine.connection import disconnect

# You may have to update this dict to match your DB credentials
credentials = {'db_user': '',
               'db_password': '',
               'db_hostname': 'localhost',
               'db_port': 27017,
               'db_authentication_database': '',
               'db_ssl_enabled': False}

uri = create_mongodb_uri_string(**credentials)

# disconnect(alias='default')
connect('smartshark_2_1', host=uri, alias='default')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

# Analise

In [3]:
projectsList = ['archiva', 'commons-math', 'ant-ivy', 'kafka','zeppelin','mahout','struts','wss4j','xerces2-j','jspwiki']
smallInfos = {}
fullInfos = {}
infoCommit = {}

for p in projectsList:
    project = Project.objects(name=p).get()

    vcs_system = VCSSystem.objects(project_id=project.id).get()

    print('VCS System:', vcs_system.url)

    num_commits = Commit.objects(vcs_system_id=vcs_system.id).count()

    print('Number of commits:', num_commits)

    issue_tracker = IssueSystem.objects(project_id=project.id).get()

    print('Issue Tracker:', issue_tracker.url)

    num_issues = Issue.objects(issue_system_id=issue_tracker.id).count()

    print('Number of issues:', num_issues)

    count_linked_issue = 0
    count_refactorings_refdiff = 0
    count_refactorings_refactoringminer = 0

    for commit in Commit.objects(vcs_system_id=vcs_system.id).only('labels', 'linked_issue_ids'):
        if commit.labels is not None and 'validated_bugfix' in commit.labels and commit.labels['validated_bugfix']==True:
            if commit.linked_issue_ids is not None and len(commit.linked_issue_ids)>0:
                count_refactorings_refdiff += Refactoring.objects(commit_id=commit.id, detection_tool='refDiff').count()
                count_refactorings_refactoringminer += Refactoring.objects(commit_id=commit.id, detection_tool='rMiner').count()
                count_linked_issue += 1
                infoCommit[p] = {commit.id : {'linked_issue_ids': commit.linked_issue_ids, 'refactorings_refdiff': Refactoring.objects(commit_id=commit.id, detection_tool='refDiff').count(), 'refactorings_refactoringminer': Refactoring.objects(commit_id=commit.id, detection_tool='rMiner').count()}}
    print('count_linked_issue:', count_linked_issue)
    print('Number of refactorings detected by refDiff:', count_refactorings_refdiff)
    print('Number of refactorings detected by RefactoringMiner:', count_refactorings_refactoringminer)
    fullInfos[p] = {'num_commits': num_commits, 'num_issues': num_issues, 'total_linked_issue': count_linked_issue, 'total_refactorings_refdiff': count_refactorings_refdiff, 'total_refactorings_refactoringminer': count_refactorings_refactoringminer, 'infoCommit': infoCommit}   
    smallInfos[p] = {'num_commits': num_commits, 'num_issues': num_issues, 'total_linked_issue': count_linked_issue, 'total_refactorings_refdiff': count_refactorings_refdiff, 'total_refactorings_refactoringminer': count_refactorings_refactoringminer}   

VCS System: https://github.com/apache/archiva.git
Number of commits: 10261
Issue Tracker: https://issues.apache.org/jira/rest/api/2/search?jql=project=ARCHIVA
Number of issues: 2010
count_linked_issue: 543
Number of refactorings detected by refDiff: 505
Number of refactorings detected by RefactoringMiner: 1100
VCS System: https://github.com/apache/commons-math
Number of commits: 7222
Issue Tracker: https://issues.apache.org/jira/rest/api/2/search?jql=project=MATH
Number of issues: 1455
count_linked_issue: 396
Number of refactorings detected by refDiff: 115
Number of refactorings detected by RefactoringMiner: 303
VCS System: https://github.com/apache/ant-ivy
Number of commits: 3189
Issue Tracker: https://issues.apache.org/jira/rest/api/2/search?jql=project=IVY
Number of issues: 1585
count_linked_issue: 568
Number of refactorings detected by refDiff: 204
Number of refactorings detected by RefactoringMiner: 389
VCS System: https://github.com/apache/kafka.git
Number of commits: 6971
Issue 

In [4]:
smallInfos

{'archiva': {'num_commits': 10261,
  'num_issues': 2010,
  'total_linked_issue': 543,
  'total_refactorings_refdiff': 505,
  'total_refactorings_refactoringminer': 1100},
 'commons-math': {'num_commits': 7222,
  'num_issues': 1455,
  'total_linked_issue': 396,
  'total_refactorings_refdiff': 115,
  'total_refactorings_refactoringminer': 303},
 'ant-ivy': {'num_commits': 3189,
  'num_issues': 1585,
  'total_linked_issue': 568,
  'total_refactorings_refdiff': 204,
  'total_refactorings_refactoringminer': 389},
 'kafka': {'num_commits': 6971,
  'num_issues': 7007,
  'total_linked_issue': 0,
  'total_refactorings_refdiff': 0,
  'total_refactorings_refactoringminer': 0},
 'zeppelin': {'num_commits': 4735,
  'num_issues': 3557,
  'total_linked_issue': 0,
  'total_refactorings_refdiff': 0,
  'total_refactorings_refactoringminer': 0},
 'mahout': {'num_commits': 4167,
  'num_issues': 2062,
  'total_linked_issue': 327,
  'total_refactorings_refdiff': 150,
  'total_refactorings_refactoringminer':

In [5]:
fullInfos

{'archiva': {'num_commits': 10261,
  'num_issues': 2010,
  'total_linked_issue': 543,
  'total_refactorings_refdiff': 505,
  'total_refactorings_refactoringminer': 1100,
  'infoCommit': {'archiva': {ObjectId('5b58546137b4503ec4b27c1b'): {'linked_issue_ids': [ObjectId('5b62ddeddddc135844b585f2')],
     'refactorings_refdiff': 0,
     'refactorings_refactoringminer': 0}},
   'commons-math': {ObjectId('5a85633a77d2b433dd719fc1'): {'linked_issue_ids': [ObjectId('5a85a157bdccb45b59be3b46')],
     'refactorings_refdiff': 0,
     'refactorings_refactoringminer': 0}},
   'ant-ivy': {ObjectId('5a82f5ea912063217d88dfc2'): {'linked_issue_ids': [ObjectId('58c8ddac1cf3f06cf159299e')],
     'refactorings_refdiff': 0,
     'refactorings_refactoringminer': 0}},
   'mahout': {ObjectId('5b1fa0c71837be5c9925f49e'): {'linked_issue_ids': [ObjectId('5b29f9dce7b4076bb2c02369')],
     'refactorings_refdiff': 0,
     'refactorings_refactoringminer': 2}},
   'wss4j': {ObjectId('5b48cf6356677a7f01f8a43f'): {'lin

In [6]:
infoCommit

{'archiva': {ObjectId('5b58546137b4503ec4b27c1b'): {'linked_issue_ids': [ObjectId('5b62ddeddddc135844b585f2')],
   'refactorings_refdiff': 0,
   'refactorings_refactoringminer': 0}},
 'commons-math': {ObjectId('5a85633a77d2b433dd719fc1'): {'linked_issue_ids': [ObjectId('5a85a157bdccb45b59be3b46')],
   'refactorings_refdiff': 0,
   'refactorings_refactoringminer': 0}},
 'ant-ivy': {ObjectId('5a82f5ea912063217d88dfc2'): {'linked_issue_ids': [ObjectId('58c8ddac1cf3f06cf159299e')],
   'refactorings_refdiff': 0,
   'refactorings_refactoringminer': 0}},
 'mahout': {ObjectId('5b1fa0c71837be5c9925f49e'): {'linked_issue_ids': [ObjectId('5b29f9dce7b4076bb2c02369')],
   'refactorings_refdiff': 0,
   'refactorings_refactoringminer': 2}},
 'wss4j': {ObjectId('5b48cf6356677a7f01f8a43f'): {'linked_issue_ids': [ObjectId('5b4db187174f92230d646075')],
   'refactorings_refdiff': 0,
   'refactorings_refactoringminer': 0}},
 'jspwiki': {ObjectId('5c38b739ca4aae332c243eab'): {'linked_issue_ids': [ObjectId('